# Growthcurves Tutorial

This notebook demonstrates how to analyze time-series growth data (OD measurements with corresponding time points) using the `growthcurves` package. You'll learn how to:

1. Fit multiple parametric and non-parametric models to growth data
2. Extract and compare growth statistics across different methods
3. Visualize growth curves with annotations showing key metrics
4. Plot instantaneous growth rates and identify exponential phase boundaries

The tutorial uses example bacterial growth data measured over ~138 hours.

In [ ]:
import growthcurves as gc
import pandas as pd
import numpy as np

# Example bacterial growth data: OD600 measurements
# This represents a typical growth curve with lag, exponential, and stationary phases
data = [
    0.0493,
    0.0494,
    0.0492,
    0.0491,
    0.0491,
    0.0492,
    0.0492,
    0.0490,
    0.0491,
    0.0491,
    0.0492,
    0.0490,
    0.0492,
    0.0490,
    0.0490,
    0.0489,
    0.0491,
    0.0491,
    0.0491,
    0.0491,
    0.0491,
    0.0491,
    0.0492,
    0.0491,
    0.0492,
    0.0492,
    0.0492,
    0.0493,
    0.0492,
    0.0493,
    0.0495,
    0.0494,
    0.0494,
    0.0494,
    0.0494,
    0.0494,
    0.0495,
    0.0494,
    0.0495,
    0.0497,
    0.0496,
    0.0496,
    0.0498,
    0.0497,
    0.0498,
    0.0498,
    0.0498,
    0.0499,
    0.0500,
    0.0499,
    0.0501,
    0.0499,
    0.0501,
    0.0500,
    0.0499,
    0.0502,
    0.0502,
    0.0503,
    0.0502,
    0.0503,
    0.0503,
    0.0503,
    0.0504,
    0.0503,
    0.0504,
    0.0506,
    0.0506,
    0.0506,
    0.0506,
    0.0507,
    0.0508,
    0.0509,
    0.0509,
    0.0509,
    0.0510,
    0.0510,
    0.0510,
    0.0512,
    0.0512,
    0.0513,
    0.0514,
    0.0513,
    0.0514,
    0.0515,
    0.0515,
    0.0516,
    0.0515,
    0.0518,
    0.0518,
    0.0518,
    0.0520,
    0.0519,
    0.0520,
    0.0522,
    0.0521,
    0.0522,
    0.0523,
    0.0523,
    0.0525,
    0.0526,
    0.0527,
    0.0528,
    0.0528,
    0.0527,
    0.0530,
    0.0531,
    0.0531,
    0.0532,
    0.0534,
    0.0533,
    0.0535,
    0.0534,
    0.0537,
    0.0537,
    0.0540,
    0.0539,
    0.0540,
    0.0540,
    0.0543,
    0.0544,
    0.0545,
    0.0546,
    0.0547,
    0.0549,
    0.0549,
    0.0551,
    0.0552,
    0.0553,
    0.0554,
    0.0555,
    0.0557,
    0.0557,
    0.0559,
    0.0560,
    0.0560,
    0.0564,
    0.0565,
    0.0567,
    0.0567,
    0.0570,
    0.0571,
    0.0574,
    0.0575,
    0.0576,
    0.0579,
    0.0581,
    0.0582,
    0.0583,
    0.0585,
    0.0588,
    0.0591,
    0.0592,
    0.0594,
    0.0598,
    0.0600,
    0.0602,
    0.0606,
    0.0610,
    0.0613,
    0.0616,
    0.0620,
    0.0624,
    0.0627,
    0.0633,
    0.0638,
    0.0643,
    0.0649,
    0.0655,
    0.0659,
    0.0666,
    0.0672,
    0.0679,
    0.0685,
    0.0695,
    0.0703,
    0.0712,
    0.0722,
    0.0732,
    0.0741,
    0.0754,
    0.0766,
    0.0780,
    0.0792,
    0.0806,
    0.0821,
    0.0834,
    0.0853,
    0.0867,
    0.0885,
    0.0900,
    0.0920,
    0.0941,
    0.0962,
    0.0982,
    0.1002,
    0.1024,
    0.1044,
    0.1063,
    0.1078,
    0.1100,
    0.1120,
    0.1141,
    0.1163,
    0.1188,
    0.1214,
    0.1238,
    0.1262,
    0.1290,
    0.1319,
    0.1346,
    0.1376,
    0.1407,
    0.1438,
    0.1468,
    0.1502,
    0.1538,
    0.1576,
    0.1615,
    0.1653,
    0.1694,
    0.1735,
    0.1783,
    0.1834,
    0.1895,
    0.1949,
    0.2008,
    0.2072,
    0.2128,
    0.2194,
    0.2255,
    0.2321,
    0.2386,
    0.2451,
    0.2523,
    0.2590,
    0.2647,
    0.2700,
    0.2752,
    0.2812,
    0.2866,
    0.2920,
    0.2969,
    0.3022,
    0.3081,
    0.3130,
    0.3187,
    0.3233,
    0.3280,
    0.3328,
    0.3363,
    0.3409,
    0.3451,
    0.3486,
    0.3532,
    0.3570,
    0.3599,
    0.3634,
    0.3670,
    0.3703,
    0.3736,
    0.3773,
    0.3806,
    0.3845,
    0.3882,
    0.3933,
    0.3992,
    0.4032,
    0.4068,
    0.4108,
    0.4146,
    0.4177,
    0.4201,
    0.4236,
    0.4263,
    0.4288,
    0.4322,
    0.4347,
    0.4373,
    0.4394,
    0.4412,
    0.4426,
    0.4440,
    0.4461,
    0.4481,
    0.4487,
    0.4500,
    0.4514,
    0.4522,
    0.4534,
    0.4532,
    0.4541,
    0.4545,
    0.4552,
    0.4553,
    0.4557,
    0.4566,
    0.4561,
    0.4571,
    0.4579,
    0.4579,
    0.4588,
    0.4587,
    0.4600,
    0.4595,
    0.4601,
    0.4596,
    0.4597,
    0.4601,
    0.4603,
    0.4598,
    0.4596,
    0.4597,
    0.4595,
    0.4600,
    0.4607,
    0.4606,
    0.4606,
    0.4601,
    0.4605,
    0.4602,
    0.4650,
    0.4646,
    0.4637,
    0.4622,
    0.4609,
    0.4605,
    0.4601,
    0.4597,
    0.4605,
    0.4600,
    0.4598,
    0.4600,
    0.4604,
    0.4602,
    0.4605,
    0.4606,
    0.4611,
    0.4604,
    0.4605,
    0.4609,
    0.4604,
    0.4615,
    0.4613,
    0.4615,
    0.4615,
    0.4616,
    0.4616,
    0.4617,
    0.4622,
    0.4624,
    0.4621,
    0.4621,
    0.4625,
    0.4626,
    0.4625,
    0.4626,
    0.4627,
    0.4623,
    0.4628,
    0.4628,
    0.4625,
    0.4633,
    0.4632,
    0.4632,
    0.4641,
    0.4644,
    0.4645,
    0.4645,
    0.4646,
    0.4652,
    0.4661,
    0.4659,
    0.4662,
    0.4666,
    0.4661,
    0.4673,
    0.4675,
    0.4678,
    0.4674,
    0.4683,
    0.4683,
    0.4684,
    0.4693,
    0.4697,
    0.4697,
    0.4702,
    0.4699,
    0.4701,
    0.4715,
    0.4716,
    0.4718,
    0.4723,
    0.4724,
    0.4724,
    0.4734,
    0.4730,
    0.4734,
    0.4744,
    0.4738,
    0.4745,
    0.4750,
    0.4758,
    0.4762,
    0.4764,
    0.4766,
    0.4771,
    0.4775,
    0.4777,
    0.4784,
    0.4783,
    0.4787,
    0.4796,
    0.4800,
    0.4805,
    0.4812,
    0.4817,
    0.4825,
    0.4826,
    0.4826,
    0.4833,
    0.4837,
    0.4843,
    0.4842,
    0.4844,
    0.4851,
    0.4860,
    0.4867,
    0.4864,
    0.4872,
    0.4876,
    0.4880,
    0.4889,
    0.4896,
    0.4893,
    0.4901,
    0.4906,
    0.4907,
    0.4913,
    0.4920,
    0.4926,
    0.4929,
    0.4926,
    0.4932,
    0.4940,
    0.4943,
    0.4949,
    0.4955,
    0.4957,
    0.4960,
    0.4968,
    0.4972,
    0.4982,
    0.4982,
    0.4988,
    0.4989,
    0.4999,
    0.5000,
    0.5006,
    0.5012,
    0.5013,
    0.5016,
    0.5017,
    0.5027,
    0.5023,
    0.5034,
    0.5034,
    0.5043,
    0.5039,
    0.5055,
    0.5053,
    0.5064,
    0.5065,
    0.5067,
    0.5070,
    0.5075,
    0.5083,
    0.5090,
    0.5091,
    0.5098,
    0.5097,
    0.5100,
    0.5109,
    0.5103,
    0.5110,
    0.5122,
    0.5123,
    0.5131,
    0.5129,
    0.5130,
    0.5139,
    0.5140,
    0.5144,
    0.5157,
    0.5158,
    0.5162,
    0.5172,
    0.5171,
    0.5170,
    0.5174,
    0.5190,
    0.5186,
    0.5193,
    0.5195,
    0.5203,
    0.5202,
    0.5211,
    0.5212,
    0.5224,
    0.5224,
    0.5227,
    0.5229,
    0.5247,
    0.5246,
    0.5251,
    0.5255,
    0.5262,
    0.5270,
    0.5273,
    0.5273,
    0.5279,
    0.5289,
    0.5290,
    0.5288,
    0.5298,
    0.5305,
    0.5309,
    0.5309,
    0.5319,
    0.5322,
    0.5325,
    0.5331,
    0.5337,
    0.5342,
    0.5344,
    0.5347,
    0.5359,
    0.5364,
    0.5361,
    0.5372,
    0.5372,
    0.5378,
    0.5381,
    0.5383,
    0.5389,
    0.5396,
    0.5401,
    0.5405,
    0.5400,
    0.5408,
    0.5417,
    0.5424,
    0.5447,
    0.5473,
    0.5485,
    0.5501,
    0.5499,
    0.5494,
    0.5493,
    0.5489,
    0.5486,
    0.5480,
    0.5475,
    0.5472,
    0.5475,
    0.5471,
    0.5464,
    0.5462,
    0.5457,
    0.5457,
    0.5452,
    0.5456,
    0.5455,
    0.5453,
    0.5445,
    0.5451,
    0.5457,
    0.5449,
    0.5447,
    0.5448,
    0.5453,
    0.5452,
    0.5450,
    0.5444,
    0.5444,
    0.5450,
    0.5451,
    0.5452,
    0.5444,
    0.5454,
    0.5455,
    0.5454,
    0.5456,
    0.5461,
    0.5466,
    0.5467,
    0.5468,
    0.5473,
    0.5471,
    0.5478,
    0.5476,
    0.5476,
    0.5483,
    0.5486,
    0.5493,
    0.5485,
    0.5488,
    0.5490,
    0.5503,
    0.5496,
    0.5497,
    0.5502,
    0.5502,
    0.5503,
    0.5500,
    0.5505,
    0.5504,
    0.5512,
    0.5514,
    0.5507,
    0.5512,
    0.5513,
    0.5517,
    0.5519,
    0.5519,
    0.5517,
    0.5523,
    0.5526,
    0.5522,
    0.5518,
    0.5527,
    0.5522,
    0.5525,
    0.5527,
    0.5529,
    0.5533,
    0.5532,
    0.5537,
    0.5533,
    0.5538,
    0.5540,
    0.5538,
    0.5538,
    0.5541,
    0.5541,
    0.5541,
    0.5545,
    0.5541,
    0.5550,
    0.5549,
    0.5555,
    0.5552,
    0.5552,
    0.5556,
    0.5554,
    0.5552,
    0.5560,
    0.5561,
    0.5558,
    0.5564,
    0.5567,
    0.5559,
    0.5567,
    0.5566,
    0.5566,
    0.5567,
    0.5560,
    0.5571,
    0.5568,
    0.5572,
    0.5569,
    0.5576,
    0.5580,
    0.5578,
    0.5575,
    0.5581,
    0.5583,
    0.5577,
    0.5578,
    0.5581,
    0.5586,
    0.5590,
    0.5586,
    0.5594,
    0.5598,
    0.5591,
    0.5597,
    0.5598,
    0.5602,
    0.5601,
    0.5604,
    0.5602,
    0.5603,
    0.5606,
    0.5608,
]

# Create time array: measurements taken every 12 minutes, converted to hours
# Total duration: ~138.6 hours
time = np.array([(12 * n) / 60 for n in range(len(data))])

## Compare Growth Statistics Across Methods

This section demonstrates how to fit multiple models to the same growth data and compare their results. We'll use both parametric models (Logistic, Gompertz, Richards, Baranyi) and non-parametric methods (Spline, Sliding Window).

### Understanding Fit Results

All fitting functions return a dictionary with the following structure:
- `model_type`: String identifier for the model/method used
- `params`: Dictionary containing model-specific parameters AND the fitting window bounds (`fit_t_min`, `fit_t_max`)

### Growth Statistics Explained

The `extract_stats_from_fit()` function calculates these key metrics:

- `max_od`: Maximum OD value within the fitted window
- `specific_growth_rate`: Maximum specific growth rate μ_max (hour⁻¹)
- `doubling_time`: Time to double the population at peak growth (hours)
- `exp_phase_start`: When exponential phase begins (hours)
- `exp_phase_end`: When exponential phase ends (hours)
- `time_at_umax`: Time when μ reaches its maximum (hours)
- `od_at_umax`: OD value at time of maximum μ
- `fit_t_min`: Start of fitting window (hours)
- `fit_t_max`: End of fitting window (hours)
- `fit_method`: Identifier for the method used
- `model_rmse`: Root mean squared error in log space (when applicable)

### Phase Boundary Detection Method

The exponential phase boundaries are determined by tracking the instantaneous **specific growth rate** μ:
- `exp_phase_start`: First time when μ exceeds 15% of μ_max (default threshold)
- `exp_phase_end`: First time after peak when μ drops below 15% of μ_max

This μ-based approach is more biologically meaningful than using absolute growth rate (dN/dt) because it normalizes by population size.

In [ ]:
# Step 1: Fit parametric models to the growth data
# These models use mathematical equations (logistic, Gompertz, etc.) to describe growth
logistic_fit = gc.parametric.fit_parametric(time, data, method="logistic")
gompertz_fit = gc.parametric.fit_parametric(time, data, method="gompertz")
richards_fit = gc.parametric.fit_parametric(time, data, method="richards")
baranyi_fit = gc.parametric.fit_parametric(time, data, method="baranyi")

# Step 2: Fit non-parametric models
# These methods don't assume a specific growth equation
spline_fit = gc.non_parametric.fit_non_parametric(time, data, method="spline")
sliding_window_fit = gc.non_parametric.fit_non_parametric(
    time, data, method="sliding_window"
)

# Step 3: Extract growth statistics from each fit
# This calculates metrics like growth rate, doubling time, phase boundaries, etc.
logistic_stats = gc.utils.extract_stats_from_fit(logistic_fit, time, data)
gompertz_stats = gc.utils.extract_stats_from_fit(gompertz_fit, time, data)
richards_stats = gc.utils.extract_stats_from_fit(richards_fit, time, data)
baranyi_stats = gc.utils.extract_stats_from_fit(baranyi_fit, time, data)
spline_stats = gc.utils.extract_stats_from_fit(spline_fit, time, data)
sliding_window_stats = gc.utils.extract_stats_from_fit(sliding_window_fit, time, data)

# Step 4: Organize results into a dictionary for easy comparison
methods = {
    "Logistic": logistic_stats,
    "Gompertz": gompertz_stats,
    "Richards": richards_stats,
    "Baranyi": baranyi_stats,
    "Spline": spline_stats,
    "Sliding window": sliding_window_stats,
}

# Extract metric names (all methods return the same set of metrics)
metrics = [k for k in next(iter(methods.values())).keys() if k != "fit_method"]

# Step 5: Create a pandas DataFrame for easy visualization
# Rows = methods, Columns = metrics
stats_df = pd.DataFrame(methods).T
stats_df = stats_df[metrics]
stats_df

## Access Fitted Parameters

Each fit result contains a `params` dictionary with the model-specific parameters. These parameters describe the fitted mathematical model and include the time window over which the fit was performed.

**Parametric models** include biological parameters like:
- Carrying capacity (K)
- Initial population (y0)
- Growth rate parameters (r, mu_max, lam, etc.)
- Fitting window bounds (fit_t_min, fit_t_max)

**Non-parametric models** include:
- Spline parameters (knots, coefficients, smoothing factor)
- Time of maximum growth rate
- Fitting window bounds

In [ ]:
# Import pretty print for nicer output formatting
from pprint import pprint

# Organize parametric and non-parametric fit results for comparison
parametric_params = {
    "Logistic": logistic_fit,
    "Gompertz": gompertz_fit,
    "Richards": richards_fit,
    "Baranyi": baranyi_fit,
}

non_parametric_params = {
    "Spline": spline_fit,
    "Sliding window": sliding_window_fit,
}

# Display the full parameter dictionaries for each model type
print("=== Parametric Model Parameters ===")
print("Each model has specific biological parameters (K, r, mu_max, etc.)")
pprint(parametric_params)

print("\n=== Non-Parametric Model Parameters ===")
print("These contain mathematical fitting parameters rather than biological ones")
pprint(non_parametric_params)

## Visualize Growth Statistics Comparison

The computed growth statistics can be compared visually using bar charts. This helps identify how different fitting methods estimate key parameters like maximum growth rate, doubling time, and phase boundaries.

The plot below shows all metrics side-by-side for each method.

In [ ]:
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Get all metric names from the DataFrame columns
metrics = list(stats_df.columns)
n_metrics = len(metrics)

# Set up subplot grid (3 columns per row)
n_cols = 3
n_rows = math.ceil(n_metrics / n_cols)

# Create subplot figure with formatted titles
fig = make_subplots(
    rows=n_rows,
    cols=n_cols,
    subplot_titles=[key.replace("_", " ").title() for key in metrics],
    horizontal_spacing=0.08,
    vertical_spacing=0.15,
)

# Get method names (row labels from DataFrame)
method_names = list(stats_df.index)

# Create a bar chart for each metric
for i, key in enumerate(metrics):
    # Calculate position in subplot grid
    row = i // n_cols + 1
    col = i % n_cols + 1
    
    # Extract values for this metric
    values = stats_df[key]
    numeric_values = values.apply(lambda v: pd.to_numeric(v, errors="coerce"))
    has_numeric = numeric_values.notna().any()

    # Add bar chart to subplot
    if has_numeric:
        fig.add_trace(
            go.Bar(
                x=method_names,
                y=numeric_values.tolist(),
                showlegend=False,
                marker=dict(line=dict(color="black", width=1)),
            ),
            row=row,
            col=col,
        )
        fig.update_yaxes(
            title_text=key.replace("_", " ").title(), automargin=True, row=row, col=col
        )
    else:
        # Handle non-numeric data (shouldn't occur with current metrics)
        fig.add_trace(
            go.Bar(
                x=method_names,
                y=[0] * len(method_names),
                showlegend=False,
                marker=dict(line=dict(color="black", width=1)),
            ),
            row=row,
            col=col,
        )
        fig.update_yaxes(showticklabels=False, automargin=True, row=row, col=col)

    # Rotate x-axis labels for readability
    fig.update_xaxes(tickangle=20, automargin=True, row=row, col=col)

# Set overall figure layout
fig.update_layout(
    title="Growth Statistics Comparison Across Methods",
    height=300 * n_rows,
    width=1100,
    margin=dict(l=80, r=30, t=80, b=80),
)

fig.show()

## Annotated Growth Curve Plots

The `growthcurves` plotting API makes it easy to visualize growth data with overlaid annotations showing key features.

### Two-Step Plotting Process

1. **`create_base_plot(time, data, scale)`**: Creates the base plot with OD data points
   - `scale` can be "linear" or "log" (for semi-log plots)

2. **`annotate_plot(fig, ...)`**: Adds annotations to the base plot
   - `phase_boundaries`: Tuple of (start, end) times for exponential phase (vertical shaded region)
   - `time_umax`: Adds vertical line at time of maximum growth rate
   - `od_umax`: Adds horizontal line at OD value when growth rate is maximum
   - `od_max`: Adds horizontal line at maximum OD
   - `umax_point`: Tuple (time, od) to mark the point of maximum growth rate
   - `fitted_model`: The fit result dictionary - automatically plots the model curve within its fitting window
   
   Any annotation can be omitted by passing `None` or simply not including it.

### Automatic Fitting Window Detection

When you pass a fit result to `fitted_model`, the function automatically reads `fit_t_min` and `fit_t_max` from the `params` dictionary. The model curve is only plotted within this window to avoid extrapolation.

In [ ]:
# Example: Creating an annotated plot with a spline fit

# Step 1: Fit a non-parametric spline model to the data
spline_fit = gc.non_parametric.fit_non_parametric(time, data, method="spline")

# Step 2: Extract growth statistics from the fit
spline_stats = gc.utils.extract_stats_from_fit(spline_fit, time, data)

# Step 3: Create base plot with log scale (linear scale also available)
fig = gc.plot.create_base_plot(time, data, scale="log")

# Step 4: Add annotations to the plot
# - Shaded region shows exponential phase (based on μ thresholds)
# - Vertical line shows time of maximum growth rate
# - Horizontal lines show OD at max growth rate and maximum OD
# - Fitted spline curve is overlaid (only within its fitting window)
fig = gc.plot.annotate_plot(
    fig,
    scale="log",  # Must match the scale used in create_base_plot
    phase_boundaries=(spline_stats["exp_phase_start"], spline_stats["exp_phase_end"]),
    time_umax=spline_stats["time_at_umax"],
    od_umax=spline_stats["od_at_umax"],
    od_max=spline_stats["max_od"],
    umax_point=(spline_stats["time_at_umax"], spline_stats["od_at_umax"]),
    fitted_model=spline_fit,  # Pass the fit result - it contains fit_t_min/max in params
)

fig.show()

## Visualizing Instantaneous Growth Rates

The `plot_derivative_metric()` function lets you visualize how growth rate changes over time. This is useful for understanding growth dynamics and validating phase boundary detection.

### Two Metrics Available

1. **Specific Growth Rate (μ)**: `metric="mu"`
   - Defined as μ = (1/N) × dN/dt
   - Represents growth rate normalized by population size
   - Units: hour⁻¹ (for time in hours)
   - **This is the metric used to determine exponential phase boundaries**

2. **Absolute Growth Rate (dN/dt)**: `metric="dndt"`
   - Raw rate of OD change over time
   - Not normalized by population size
   - Units: OD/hour (for time in hours)

### Plot Components

Each plot shows three traces:
1. **Light grey line**: Metric calculated from raw, unsmoothed data (noisy)
2. **Pink/red line**: Smoothed metric using Savitzky-Golay filtering (recommended for analysis)
3. **Dashed blue line**: Metric derived from the fitted model (if `fit_result` is provided)

### Phase Boundary Visualization

When you provide `phase_boundaries`, a shaded region shows the exponential phase. These boundaries are calculated based on the **specific growth rate (μ)**, specifically when μ crosses 15% of μ_max (default threshold).

**Why use μ instead of dN/dt for phase detection?**  
The specific growth rate μ normalizes by population size, making it more biologically meaningful. A small population growing quickly has a low dN/dt but high μ, while a large population growing slowly has high dN/dt but low μ.

In [ ]:
# Plot the specific growth rate (μ) over time
# This shows how the population's growth rate (normalized by size) changes
fig_mu = gc.plot.plot_derivative_metric(
    time,
    data,
    metric="mu",  # Specific growth rate: μ = (1/N) × dN/dt
    fit_result=gompertz_fit,  # Include model-derived μ curve
    phase_boundaries=(
        gompertz_stats["exp_phase_start"],
        gompertz_stats["exp_phase_end"],
    ),
    title="Specific Growth Rate (μ) - Gompertz Fit",
)

fig_mu.show()

In [ ]:
# Plot the absolute growth rate (dN/dt) over time
# This shows the raw rate of OD change (not normalized by population size)
fig_dndt = gc.plot.plot_derivative_metric(
    time,
    data,
    metric="dndt",  # Absolute growth rate: dOD/dt
    fit_result=gompertz_fit,  # Include model-derived dN/dt curve
    phase_boundaries=(
        gompertz_stats["exp_phase_start"],
        gompertz_stats["exp_phase_end"],
    ),
    title="Absolute Growth Rate (dOD/dt) - Gompertz Fit",
)

fig_dndt.show()

## Complete Analysis Example: Parametric Gompertz Fit

This section shows a complete workflow using a parametric Gompertz model:
1. Fit the model to data
2. Extract growth statistics
3. Create annotated plots showing the growth curve
4. Visualize instantaneous growth rates

The Gompertz model is particularly well-suited for bacterial growth curves as it naturally captures asymmetric growth with a slower approach to stationary phase.

In [ ]:
# Fit a parametric Gompertz model to the growth data
# Returns a dictionary with model_type and params (including K, mu_max, lam, y0, fit_t_min, fit_t_max)
gompertz_fit = gc.parametric.fit_parametric(time, data, method="gompertz")

# Extract growth statistics from the fitted model
# Returns metrics like max_od, specific_growth_rate, doubling_time, phase boundaries, etc.
gompertz_stats = gc.utils.extract_stats_from_fit(gompertz_fit, time, data)

In [ ]:
# Create an annotated growth curve plot
# You can use either "linear" or "log" scale

scale = "linear"  # Change to "log" for semi-log plot

# Step 1: Create base plot with the chosen scale
fig_log = gc.plot.create_base_plot(time, data, scale=scale)

# Step 2: Add annotations (all optional)
# - Exponential phase shown as shaded region (based on μ thresholds)
# - Vertical line at time of max growth rate
# - Horizontal line at OD when growth rate is maximum
# - Horizontal line at maximum OD
# - Point marker at maximum growth rate
# - Fitted Gompertz curve (automatically uses fit_t_min/max from params)
fig_log = gc.plot.annotate_plot(
    fig_log,
    scale=scale,  # Must match the base plot scale
    phase_boundaries=(
        gompertz_stats["exp_phase_start"],
        gompertz_stats["exp_phase_end"],
    ),
    time_umax=gompertz_stats["time_at_umax"],
    od_umax=gompertz_stats["od_at_umax"],
    od_max=gompertz_stats["max_od"],
    umax_point=(gompertz_stats["time_at_umax"], gompertz_stats["od_at_umax"]),
    fitted_model=gompertz_fit,
)

fig_log.show()

In [ ]:
# Visualize the specific growth rate (μ) for the Gompertz fit
# This plot shows:
# - Raw μ calculated from data (light grey, noisy)
# - Smoothed μ using Savitzky-Golay filter (pink/red)
# - Model-derived μ from Gompertz equation (dashed blue)
# - Exponential phase boundaries (shaded region)

fig_mu = gc.plot.plot_derivative_metric(
    time,
    data,
    metric="mu",  # Specific growth rate
    fit_result=gompertz_fit,
    phase_boundaries=(
        gompertz_stats["exp_phase_start"],
        gompertz_stats["exp_phase_end"],
    ),
    title="Specific Growth Rate (μ) - Gompertz Fit",
)

fig_mu.show()

In [ ]:
# Visualize the absolute growth rate (dN/dt) for the Gompertz fit
# This plot shows:
# - Raw dN/dt calculated from data (light grey, noisy)
# - Smoothed dN/dt using Savitzky-Golay filter (pink/red)
# - Model-derived dN/dt from Gompertz equation (dashed blue)
# - Exponential phase boundaries (shaded region, based on μ not dN/dt)

fig_dndt = gc.plot.plot_derivative_metric(
    time,
    data,
    metric="dndt",  # Absolute growth rate (not normalized)
    fit_result=gompertz_fit,
    phase_boundaries=(
        gompertz_stats["exp_phase_start"],
        gompertz_stats["exp_phase_end"],
    ),
    title="Absolute Growth Rate (dN/dt) - Gompertz Fit",
)

fig_dndt.show()